In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/home/avsngh/Mathematical Statistics/Masters-Projects/Unsupervised-Learning/Data/steam_cleaned.csv')

In [6]:
df

,Name,Price,Release_date,Review_no,Review_type,Tags,Description
0,Black Myth: Wukong,59.99,2024-08-20,574097.0,Overwhelmingly Positive,"Mythology,Action RPG,Action,Souls-like,RPG,Com...",Black Myth: Wukong is an action RPG rooted in ...
1,"Warhammer 40,000: Space Marine 2",NaN,2024-09-09,23591.0,Very Positive,"Warhammer 40K,Action,Adventure,Third-Person Sh...",Embody the superhuman skill and brutality of a...
2,Counter-Strike 2,0.00,2012-08-21,8286153.0,Very Positive,"FPS,Shooter,Multiplayer,Competitive,Action,Tea...","For over two decades, Counter-Strike has offer..."
3,Warframe,0.00,NaN,589527.0,Very Positive,"Free to Play,Action RPG,Looter Shooter,Third-P...",Awaken as an unstoppable warrior and battle al...
4,Grand Theft Auto V,10.48,2015-04-14,1703156.0,Very Positive,"Open World,Action,Multiplayer,Crime,Automobile...",Grand Theft Auto V for PC offers players the o...
...,...,...,...,...,...,...,...
7807,Monster Hunter World: Iceborne - MHW:I Monster...,2.99,2020-02-06,39.0,Positive,Action,A monster figure you can use to decorate your ...
7808,Gene Shift Auto: Deluxe Edition,8.99,2022-11-28,16.0,Positive,"Indie,Action,Free to Play,Battle Royale,Roguel...",Gene Shift Auto is a roguelike-inspired battle...
7809,Run Ralph Run,0.45,2021-03-03,26.0,Mostly Positive,"Adventure,Action,Puzzle,Arcade,Platformer,Shoo...","Ralph is a smart dinosaur, and a great shooter."
7810,Quadroids,6.19,2024-02-22,15.0,Positive,"Precision Platformer,Puzzle Platformer,2D Plat...",Quadroids is a single-player puzzle platformer...


In [5]:
print(df.isnull().sum())

Name              0
Price            51
Release_date     19
Review_no        47
Review_type      47
Tags              0
Description     426
dtype: int64


In [7]:
df.describe()

,Price,Review_no
count,7761.000000,7.765000e+03
mean,8.057703,8.875158e+03
std,10.131217,1.117250e+05
min,0.000000,1.000000e+01
25%,2.490000,5.900000e+01
50%,5.490000,2.700000e+02
75%,8.490000,1.719000e+03
max,139.990000,8.286153e+06


In [8]:
print(df.describe())


             Price     Review_no
count  7761.000000  7.765000e+03
mean      8.057703  8.875158e+03
std      10.131217  1.117250e+05
min       0.000000  1.000000e+01
25%       2.490000  5.900000e+01
50%       5.490000  2.700000e+02
75%       8.490000  1.719000e+03
max     139.990000  8.286153e+06


In [16]:
count = df[df['Price'] > 70].shape[0]
count

6